In [1]:
import numpy as np
import pandas as pd
from PIL import Image
import json 
import cv2

In [4]:
path='./dataset/gen_img_data.json'

In [5]:
with open(path, 'r') as f:
  datastore = json.load(f)

In [6]:
print(len(datastore))

23702


In [7]:
def get_values_class(json_data):
    labels=[]
    pixel_values=[]
    for i in range(len(json_data)):
    
        for name, value in json_data[i].items():    # for name, age in dictionary.iteritems():  (for Python 2.x)
            labels.append(name)
      # print(name)

            for name, value in value.items():
                pixel_values.append(value)
        # print(len(value))
        # print(value)

    return (pixel_values, labels)

In [8]:
X,y=get_values_class(datastore)

In [9]:
df=pd.DataFrame(list(zip(X,y)),columns =['X', 'labels'])

In [11]:
X_data = []

for i in range(len(df['X'])):
    image=df['X'][i]
    image=np.asarray(image)

    if image.shape[0]==1024:
    # image.shape=(32,32)
        df['X'][i] = image.reshape((32,32)).astype('float32')
    # image = cv2.resize(image, dsize=(28,28), interpolation=cv2.INTER_CUBIC)

    elif image.shape[0]==784:

        image = image.reshape((28,28)).astype('float32')

        df['X'][i] =  cv2.resize(image, dsize=(32,32), interpolation=cv2.INTER_CUBIC)

    # image = image.reshape((28,28)).astype('float32')

    # image.shape=(28,28)

  # X_data[i]=image
  

In [12]:
for i in range(len(df['X'])):
    image=df['X'][i]
    name="./dataset/"+str(i)+'.png'
    cv2.imwrite(name,image)

In [13]:
import cv2
import glob
images = np.asarray([cv2.imread(file) for file in glob.glob('./dataset/*.png')])

In [14]:
image_np_array=np.zeros((len(images),32,32,3),dtype=float)

In [15]:
image_np_array.shape

(23702, 32, 32, 3)

In [16]:
images.shape

(23702, 32, 32, 3)

In [18]:
for i in range(len(images)):

    for x in range(images[i].shape[0]):
        for y in range(images[i].shape[1]):
            for z in range(images[i].shape[2]):
                image_np_array[i][x][y][z]=images[i][x][y][z]

In [19]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(df['labels'])

LabelEncoder()

In [20]:
y_transform=le.transform(df['labels'])

In [21]:
from tensorflow.keras import utils as np_utils
Y_train = np_utils.to_categorical(y_transform, num_classes = 6)

In [22]:
from sklearn.model_selection import train_test_split

In [23]:
trainX, testX, trainY, testY = train_test_split(image_np_array,Y_train, test_size=0.2, random_state=42)

In [34]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D,Flatten,Dropout,MaxPool2D 

In [39]:
model = Sequential()
model.add(Conv2D(input_shape=(32,32,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
# model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
# model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
# model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
# model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
# model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
# model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
# model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
# model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
# model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Flatten())
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=6, activation="softmax"))

In [40]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 32, 32, 64)        1792      
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 32, 32, 64)        36928     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 8, 8, 256)         147712    
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 4, 4, 256)         0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 4, 4, 512)        

In [41]:
import keras
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

In [42]:
batch_size = 128
num_epoch = 10
#model training
model_log = model.fit(trainX, trainY,
          batch_size=batch_size,
          epochs=num_epoch,
          verbose=1,
          validation_data=( testX,  testY))

Train on 18961 samples, validate on 4741 samples
Epoch 1/10
18961/18961 [==============================] - 145s 8ms/step - loss: 4.5683 - accuracy: 0.2476 - val_loss: 1.5789 - val_accuracy: 0.2620
Epoch 2/10
18961/18961 [==============================] - 143s 8ms/step - loss: 1.5909 - accuracy: 0.2546 - val_loss: 1.5783 - val_accuracy: 0.2620
Epoch 3/10
18961/18961 [==============================] - 142s 8ms/step - loss: 1.5897 - accuracy: 0.2488 - val_loss: 1.5787 - val_accuracy: 0.2453
Epoch 4/10
18961/18961 [==============================] - 142s 7ms/step - loss: 1.5885 - accuracy: 0.2510 - val_loss: 1.5792 - val_accuracy: 0.2620
Epoch 5/10
18961/18961 [==============================] - 148s 8ms/step - loss: 1.5898 - accuracy: 0.2562 - val_loss: 1.5810 - val_accuracy: 0.2453
Epoch 6/10
18961/18961 [==============================] - 150s 8ms/step - loss: 1.5892 - accuracy: 0.2530 - val_loss: 1.5778 - val_accuracy: 0.2620
Epoch 7/10
18961/18961 [==============================] - 154s 